# Master Thesis 2nd Version of a STT Model with new State of the Art: Whisper

**Author**: Karin Thommen

**Date**: April 2023


---

**Content of the Notebook**:  Fine-tuning and Training of OpenAi Whisper ASR Model

---
**References**:
- https://huggingface.co/blog/fine-tune-whisper
- https://github.com/vasistalodagala/whisper-finetune

## Step 1: Import and Setup

In [ ]:
%%capture
!pip install datasets
!pip install transformers==4.28.0
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install audio-metadata
!pip install "dill<0.3.5"
!pip install git-lfs

In [ ]:
import pandas as pd
import os
import transformers

from datasets.fingerprint import Hasher
import pickle
import dill

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json

import IPython.display as ipd
import numpy as np
import random

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import audio_metadata

from datasets import load_dataset, Audio, load_metric, load_from_disk, DatasetDict, list_datasets
from datasets import Dataset, Sequence

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from transformers import WhisperTokenizer
from transformers import WhisperTokenizerFast
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from huggingface_hub import notebook_login

from google.colab import drive

In [ ]:
transformers.__version__

'4.28.0'

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Step 2: Load Data

In [ ]:
# login to huggingface account for data
notebook_login()

In [ ]:
print(list_datasets())

    list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.


['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity', 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'americas_nli', 'ami', 'amttl', 'anli', 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19', 'ar_res_reviews', 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect', 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset', 'ascent_kb', 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic', 'autshumato', 'facebook/babi_qa', 'banking77', 'bbaw_egyptian', 'bbc_hindi_nli', 'bc2gm_corpus', 'beans', 'best2009', 'bianet', 'bible_para', 'big_patent', 'billsum', 'bing_coronavirus_query_set', 'biomrc', 'biosses', 'blbooks', 'blbooksgenre', 'blended_skill_talk', 'blimp', 'blog_authorship_corpus', 'bn_hate_speech', 'bnl_newspapers', 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec', 'break_data', 'brwac', 'bsd_ja_en'

In [ ]:
# load dataset from huggingface (after uploading it via local machine to huggingface)
dataset = load_dataset("karinthommen/sds200")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/135271 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3638 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3636 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/karinthommen___parquet/karinthommen--sds200-a1893d366d27240a/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# check if data loading worked
dataset["train"][0]

{'audio': {'path': '09966c7743291ccf1129c8136143bf5a6132947fe352795bc6d5456a3afeb4de.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.58690691e-05, -6.36559753e-06, -1.80013558e-05]),
  'sampling_rate': 32000},
 'transcription': 'Dadurch wird auch der Lebensraum von vielen Tier- und Pflanzenarten zerstört.',
 'canton': None,
 'duration': 6.732}

In [ ]:
dataset.shape

{'train': (135271, 4), 'validation': (3638, 4), 'test': (3636, 4)}

In [ ]:
dataset["train"] = dataset["train"].filter(lambda example: example["duration"] <= 6)

Filter:   0%|          | 0/135271 [00:00<?, ? examples/s]

In [ ]:
# make dataset the same size (compared to spontaneous speech)
dataset["train"] = dataset["train"].shuffle(seed=42)
dataset["validation"] = dataset["validation"].shuffle(seed=42)
dataset["train"] = dataset["train"].select(i for i in range(0, 1900))
dataset["validation"] = dataset["validation"].select(i for i in range(0, 300))


dataset.shape # check shape

{'train': (1900, 4), 'validation': (300, 4), 'test': (3636, 4)}

In [ ]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\$\(\)\'\$\*\$\_\'̈\’\•\‹\₂\›\–\²\½\‑\°\`\&\(\)\*\+\/\=\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\[\]]'

def preprocess(batch):
  batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
  batch["transcription"] = batch["transcription"].strip()
  return batch

dataset = dataset.map(preprocess, num_proc=2)

In [ ]:
# remove columns from dataset that we do not need at the moment
dataset = dataset.remove_columns(["canton"])

In [ ]:
# filter data to delete empty samples
dataset = dataset.filter(lambda example: len(example["transcription"])!=0)

Filter:   0%|          | 0/1900 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3636 [00:00<?, ? examples/s]

In [ ]:
# load tokenizer form Whisper Tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

In [ ]:
# load tokenizer form Whisper Tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
# load feature extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
# load processor from Whisper Processor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

## Step 3: Prepare Dataset and convert it into the correct Format

In [ ]:
# downsample dataset to a sampling rate of 16kHz for the model
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# Check if audio loading worked with a random audio and sentence
rand_int = random.randint(0, len(dataset["train"])-1)
print(dataset["train"]["transcription"][rand_int])
ipd.Audio(data=dataset["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

Die Rote Karte wäre unabdingbar gewesen.


In [ ]:
# Check sentence, input array shape and sampling rate
rand_int = random.randint(0, len(dataset["train"])-1)

print("Target text:", dataset["train"][rand_int]["transcription"])
print("Input array shape:", dataset["train"][rand_int]["audio"]["array"].shape)
print("Sampling rate:", dataset["train"][rand_int]["audio"]["sampling_rate"])

Target text: Er ist also keineswegs ungültig.
Input array shape: (57600,)
Sampling rate: 16000


In [ ]:
# show sentence decoded with the special characters ( in the format that is needed by whisper )
input_str = dataset["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Die Inflationsrate erreichte im Oktober 5,5 Prozent.
Decoded w/ special:    <|startoftranscript|><|notimestamps|>Die Inflationsrate erreichte im Oktober 5,5 Prozent.<|endoftext|>
Decoded w/out special: Die Inflationsrate erreichte im Oktober 5,5 Prozent.
Are equal:             True


In [ ]:
# show format of train dataset
dataset["train"][0]

{'audio': {'path': '11008817371abcc3496370eb73d8bf1cef549ab493bb19c941f3f8decfd071fe.mp3',
  'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         7.38668859e-09, 1.81796622e-08, 2.02794581e-09]),
  'sampling_rate': 16000},
 'transcription': 'Die Inflationsrate erreichte im Oktober 5,5 Prozent.',
 'duration': 4.86}

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, num_proc=2)

Map (num_proc=2):   0%|          | 0/1900 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/300 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3636 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.shuffle(seed=500)

# make sure that there is no empty transcription in dataset
dataset = dataset.filter(lambda example: len(example["transcription"])!=0)

Filter:   0%|          | 0/1900 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3636 [00:00<?, ? examples/s]

## Fine-Tune & Train Model

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-V2-default-small",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    #per_device_eval_batch_size=8,
    predict_with_generate=True,
    #generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=100,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/karinthommen/whisper-V2-default-small into local empty directory.


In [ ]:
processor.save_pretrained(training_args.output_dir)

## Training Process

### Version 2.3 Small default whisper prepared speech

In [ ]:
trainer.train()

    This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,3.554700,1.733381,72.105938
200,1.008400,0.864410,727.381461
300,0.351500,0.842928,1038.445109
400,0.197000,0.868700,869.329346
500,0.117100,0.862771,1230.072619


TrainOutput(global_step=500, training_loss=1.0457527885437012, metrics={'train_runtime': 3196.8687, 'train_samples_per_second': 2.502, 'train_steps_per_second': 0.156, 'total_flos': 2.30406583615488e+18, 'train_loss': 1.0457527885437012, 'epoch': 4.2})

### Version 2.2 with 400 stepsize - interrupted

In [ ]:
trainer.train()

    This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
400,1.548300,1.699449,69.292474
800,1.321200,1.388738,61.126552


### Version 2.1 with 100 stepsize (large data set)

In [ ]:
trainer.train()

    This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,0.726700,0.797830,45.892411
200,0.594100,0.723930,45.558175
300,0.595600,0.684671,47.271576
400,0.591200,0.660416,47.704324
500,0.583000,0.650122,48.232066


TrainOutput(global_step=500, training_loss=0.8410588340759277, metrics={'train_runtime': 10622.8643, 'train_samples_per_second': 0.753, 'train_steps_per_second': 0.047, 'total_flos': 2.30868320256e+18, 'train_loss': 0.8410588340759277, 'epoch': 0.07})

In [ ]:
trainer.push_to_hub()